In [249]:
import pandas as pd
pd.options.display.max_colwidth = 100
import plotly.express as px
import re
import datetime
import numpy as np
import json
from Project.Database import Db
from Project._04TPMAlgorithm.transform_for_TPM_algorithm import light_location_dict

In [250]:
level1_json = json.load(
    open(Db.get_project_path("Project/_04TPMAlgorithm/TPM/output/NZERTF_year1_minsup0.14_minconf_0.5/level1.json")))

In [251]:
level2_json = json.load(
    open(Db.get_project_path("Project/_04TPMAlgorithm/TPM/output/NZERTF_year1_minsup0.14_minconf_0.5/level2.json")))

In [252]:
level3_json = json.load(
    open(Db.get_project_path("Project/_04TPMAlgorithm/TPM/output/NZERTF_year1_minsup0.14_minconf_0.5/level3.json")))

In [253]:
meta = Db.load_data(meta=True, consumption=False, hourly=False)

In [254]:
def json_to_dataframe(json_file):
    if "," in json_file[0]["name_node"]:
        level_df = pd.DataFrame(columns=["pattern","supp","conf", "time"])
        for i in json_file:
            for j in i["patterns"]:
                level_df.loc[level_df.shape[0]] = j
    else:
        level_df = pd.DataFrame(columns=["name_node","supp","conf"])
        for i in json_file:
            level_df.loc[level_df.shape[0]] = i
    return level_df

In [255]:
level2_df = json_to_dataframe(level2_json)
level2_df

,pattern,supp,conf,time
0,Load_StatusMBALights->Load_StatusMBALights,0.849315,0.856354,"{'199': [[['2014-01-16 03:30:19', '2014-01-16 03:33:19'], ['2014-01-16 03:59:19', '2014-01-16 04..."
1,Load_StatusMBALights->Load_StatusSensHeatPrntBDOWN,0.980822,0.988950,"{'199': [[['2014-01-16 06:29:19', '2014-01-16 06:44:18'], ['2014-01-16 06:59:18', '2014-01-16 08..."
2,Load_StatusMBALights>Load_StatusSensHeatPrntBDOWN,0.704110,0.709945,"{'199': [[['2014-01-16 05:59:19', '2014-01-16 06:14:19'], ['2014-01-16 05:59:19', '2014-01-16 06..."
3,Load_StatusMBALights>Load_StatusPlugLoadBlender,0.701370,0.707182,"{'199': [[['2014-01-16 05:59:19', '2014-01-16 06:14:19'], ['2014-01-16 06:04:19', '2014-01-16 06..."
4,Load_StatusMBALights>Load_StatusPlugLoadCoffeeMaker,0.701370,0.707182,"{'199': [[['2014-01-16 05:59:19', '2014-01-16 06:14:19'], ['2014-01-16 06:09:19', '2014-01-16 06..."
...,...,...,...,...
689,Load_StatusApplianceOven->Load_StatusClothesWasher,0.276712,0.651613,"{'341': [[['2014-06-07 08:13:20', '2014-06-07 10:50:20'], ['2014-06-07 10:59:20', '2014-06-07 11..."
690,Load_StatusClothesWasher>Load_StatusDryerPowerTotal,0.405479,0.896970,"{'341': [[['2014-06-07 14:29:20', '2014-06-07 15:29:20'], ['2014-06-07 14:59:20', '2014-06-07 15..."
691,Load_StatusClothesWasher|Load_StatusPlugLoadLRBlueRay,0.268493,0.632258,"{'341': [[['2014-06-07 14:29:20', '2014-06-07 15:29:20'], ['2014-06-07 14:58:20', '2014-06-07 16..."
692,Load_StatusPlugLoadLRBlueRay>Load_StatusDryerPowerTotal,0.279452,0.618182,"{'341': [[['2014-06-07 14:58:20', '2014-06-07 16:28:20'], ['2014-06-07 14:59:20', '2014-06-07 15..."


In [256]:
level3_df = json_to_dataframe(level3_json)
level3_df

,pattern,supp,conf,time
0,Load_StatusMBALights->Load_StatusMBALights*Load_StatusMBALights->Load_StatusMasterBedroomLights*...,0.698630,0.704420,"{'143': [[['2013-11-21 05:59:11', '2013-11-21 06:14:11'], ['2013-11-21 06:29:11', '2013-11-21 06..."
1,Load_StatusMBALights->Load_StatusMBALights*Load_StatusMBALights>Load_StatusBA2Lights*Load_Status...,0.701370,0.707182,"{'143': [[['2013-11-21 05:59:11', '2013-11-21 06:14:11'], ['2013-11-21 06:29:11', '2013-11-21 06..."
2,Load_StatusMBALights->Load_StatusMBALights*Load_StatusMBALights>Load_StatusSensHeatPrntBUP*Load_...,0.701370,0.701370,"{'143': [[['2013-11-21 05:59:11', '2013-11-21 06:14:11'], ['2013-11-21 06:29:11', '2013-11-21 06..."
3,Load_StatusMBALights->Load_StatusMBALights*Load_StatusMBALights>Load_StatusBR2Lights*Load_Status...,0.701370,0.707182,"{'143': [[['2013-11-21 05:59:11', '2013-11-21 06:14:11'], ['2013-11-21 06:29:11', '2013-11-21 06..."
4,Load_StatusMBALights->Load_StatusMBALights*Load_StatusMBALights>Load_StatusPlugLoadHairDryerCurl...,0.701370,0.707182,"{'143': [[['2013-11-21 05:59:11', '2013-11-21 06:14:11'], ['2013-11-21 06:29:11', '2013-11-21 06..."
...,...,...,...,...
5084,Load_StatusPlugLoadHeatingPad>Load_StatusMBALights*Load_StatusMBALights->Load_StatusMBALights*Lo...,0.701370,0.707182,"{'143': [[['2013-11-21 21:59:09', '2013-11-21 22:09:09'], ['2013-11-21 22:04:09', '2013-11-21 22..."
5085,Load_StatusPlugLoadHeatingPad>Load_StatusMBALights*Load_StatusMBALights->Load_StatusSensHeatPrnt...,0.701370,0.701370,"{'143': [[['2013-11-21 21:59:09', '2013-11-21 22:09:09'], ['2013-11-21 22:04:09', '2013-11-21 22..."
5086,Load_StatusPlugLoadHeatingPad->Load_StatusSensHeatPrntAUP*Load_StatusSensHeatPrntAUP>Load_Status...,0.698630,0.704420,"{'143': [[['2013-11-21 21:59:09', '2013-11-21 22:09:09'], ['2013-11-21 22:29:09', '2013-11-21 23..."
5087,Load_StatusApplianceOven>Load_StatusPlugLoadVideoGame*Load_StatusPlugLoadVideoGame->Load_StatusC...,0.276712,0.651613,"{'341': [[['2014-06-07 08:13:20', '2014-06-07 10:50:20'], ['2014-06-07 09:58:20', '2014-06-07 10..."


In [296]:
def filter_rule_indexes(dataframe):
    return_dict = {
        'multi_floor': {},
        'single_floor': {}
    }
    level_3_check = len(re.findall('\*', dataframe.loc[0, 'pattern'])) > 0
    #filter follows rules in dataframe for level3: max 1 and level2: 0
    if level_3_check:
        dataframe = dataframe.loc[dataframe['pattern'].str.findall('-').map(len) <= 1]
    else:
        dataframe = dataframe.loc[dataframe['pattern'].str.findall('-').map(len) == 0]

    appliance_appliance_appliance_rule_indexes_multi = []
    person_appliance_appliance_rule_indexes_multi = []
    person_person_appliance_rule_indexes_multi = []
    appliance_appliance_appliance_rule_indexes_single = []
    person_appliance_appliance_rule_indexes_single = []
    person_person_appliance_rule_indexes_single = []

    for index, row in dataframe.iterrows():
        tmp_floor_set = set()
        appliance_check_list = list()
        person_check_list = list()
        for col in set(re.findall('[\w_]+', row['pattern'])):
            person_check_list.append('SensHeat' in col)
            appliance_check_list.append('SensHeat' not in col)
            try:
                tmp_floor_set.add(meta.loc[col, 'Measurement_Floor'])
            except:
                tmp_floor_set.add(light_location_dict(meta)[col][0])

        if '1stFloor' in tmp_floor_set and '2ndFloor' in tmp_floor_set and sum(appliance_check_list) >= 1:
            if sum(person_check_list) == 1:
                person_appliance_appliance_rule_indexes_multi.append(index)
            elif sum(person_check_list) == 2:
                person_person_appliance_rule_indexes_multi.append(index)
            elif sum(person_check_list) == 0:
                appliance_appliance_appliance_rule_indexes_multi.append(index)
        else:
            if sum(person_check_list) == 1:
                person_appliance_appliance_rule_indexes_single.append(index)
            elif sum(person_check_list) == 2:
                person_person_appliance_rule_indexes_single.append(index)
            elif sum(person_check_list) == 0:
                appliance_appliance_appliance_rule_indexes_single.append(index)


    if level_3_check:
        return_dict['multi_floor'].update({
            'psn_app_app': person_appliance_appliance_rule_indexes_multi,
            'psn_psn_app': person_person_appliance_rule_indexes_multi,
            'app_app_app': appliance_appliance_appliance_rule_indexes_multi
        })
        return_dict['single_floor'].update({
            'psn_app_app': person_appliance_appliance_rule_indexes_single,
            'psn_psn_app': person_person_appliance_rule_indexes_single,
            'app_app_app': appliance_appliance_appliance_rule_indexes_single
        })
    else:
        return_dict['multi_floor'].update({
            'psn_app': person_appliance_appliance_rule_indexes_multi,
            'app_app': appliance_appliance_appliance_rule_indexes_multi
        })
        return_dict['single_floor'].update({
            'psn_app': person_appliance_appliance_rule_indexes_single,
            'app_app': appliance_appliance_appliance_rule_indexes_single
        })

    return return_dict

In [297]:
level2_index_dict = filter_rule_indexes(level2_df)
level2_index_dict

{'multi_floor': {'psn_app': [2,
   33,
   38,
   44,
   51,
   54,
   69,
   70,
   156,
   213,
   253,
   264,
   265,
   269,
   270,
   274,
   278,
   282,
   283,
   284,
   293,
   294,
   301,
   302,
   334,
   341,
   343,
   368,
   370,
   389,
   390,
   392,
   408,
   413,
   417,
   482,
   483,
   485,
   487,
   489,
   496,
   502,
   504,
   505,
   508,
   510,
   511,
   520,
   521,
   523,
   526,
   530,
   542,
   544,
   545,
   584,
   588,
   589,
   608,
   609,
   623,
   630,
   631,
   635,
   636,
   647,
   648,
   650,
   652,
   654,
   661,
   667,
   668],
  'app_app': [3,
   4,
   23,
   26,
   27,
   137,
   151,
   153,
   155,
   207,
   210,
   226,
   245,
   250,
   254,
   255,
   256,
   257,
   266,
   267,
   339,
   350,
   351,
   366,
   378,
   379,
   380,
   384,
   554,
   568,
   587,
   595,
   598,
   599,
   600,
   607,
   616,
   617,
   619,
   621,
   624,
   625,
   633,
   634,
   639,
   640,
   678,
   693]},
 'single

In [272]:
level3_index_dict = filter_rule_indexes(level3_df)

In [298]:
def redundancy_filter_tool(dataframe=level3_df, regex_str='BR3[\w]*>[\w]*ChildB', select_index_dict='psn_app_app', multi_floor=True):
    multi_floor_selector = 'multi_floor' if multi_floor else 'single_floor'
    dataframe = dataframe.loc[filter_rule_indexes(dataframe)[multi_floor_selector][select_index_dict]]
    return dataframe.loc[dataframe['pattern'].str.findall(regex_str).map(len) > 0]

In [300]:
redundancy_filter_tool(dataframe=level2_df, regex_str='[\w]*>[\w]*SensHeat', select_index_dict='psn_app', multi_floor=False)

,pattern,supp,conf,time
14,Load_StatusMBALights>Load_StatusSensHeatPrntBUP,0.841096,0.841096,"{'199': [[['2014-01-16 06:29:19', '2014-01-16 06:44:18'], ['2014-01-16 06:29:19', '2014-01-16 06..."
25,Load_StatusMBALights>Load_StatusSensHeatPrntAUP,0.556164,0.556164,"{'299': [[['2014-04-26 19:58:35', '2014-04-26 20:13:35'], ['2014-04-26 19:58:35', '2014-04-26 20..."
80,Load_StatusKitchenLights>Load_StatusSensHeatPrntBDOWN,0.986301,0.994475,"{'199': [[['2014-01-16 05:59:19', '2014-01-16 08:24:18'], ['2014-01-16 05:59:19', '2014-01-16 06..."
93,Load_StatusKitchenLights>Load_StatusSensHeatPrntADOWN,0.983562,0.991713,"{'199': [[['2014-01-16 05:59:19', '2014-01-16 08:24:18'], ['2014-01-16 06:29:19', '2014-01-16 08..."
95,Load_StatusKitchenLights>Load_StatusSensHeatPrntBUP,0.701370,0.701370,"{'199': [[['2014-01-16 05:59:19', '2014-01-16 08:24:18'], ['2014-01-16 06:29:19', '2014-01-16 06..."
104,Load_StatusKitchenLights>Load_StatusSensHeatChildADOWN,0.991781,1.000000,"{'199': [[['2014-01-16 05:59:19', '2014-01-16 08:24:18'], ['2014-01-16 06:59:18', '2014-01-16 07..."
106,Load_StatusKitchenLights>Load_StatusSensHeatChildBDOWN,0.991781,1.000000,"{'199': [[['2014-01-16 05:59:19', '2014-01-16 08:24:18'], ['2014-01-16 06:59:18', '2014-01-16 07..."
112,Load_StatusKitchenLights>Load_StatusSensHeatPrntAUP,0.983562,0.983562,"{'199': [[['2014-01-16 17:59:45', '2014-01-16 18:29:45'], ['2014-01-16 17:59:45', '2014-01-16 18..."
157,Load_StatusPlugLoadMBRTV>Load_StatusSensHeatPrntBUP,0.843836,0.843836,"{'199': [[['2014-01-16 06:14:19', '2014-01-16 06:59:18'], ['2014-01-16 06:29:19', '2014-01-16 06..."
229,Load_StatusBA2Lights>Load_StatusSensHeatPrntBUP,0.701370,0.701370,"{'199': [[['2014-01-16 06:29:19', '2014-01-16 06:49:18'], ['2014-01-16 06:29:19', '2014-01-16 06..."


In [263]:
redundancy_filter_tool(dataframe=level2_df, regex_str='SensHeat[\w]*>[\w]*', select_index_dict='psn_app')

,pattern,supp,conf,time
33,Load_StatusSensHeatPrntBDOWN>Load_StatusMBALights,0.991781,1.000000,"{'199': [[['2014-01-16 05:59:19', '2014-01-16 06:29:19'], ['2014-01-16 05:59:19', '2014-01-16 06..."
38,Load_StatusSensHeatPrntBDOWN>Load_StatusPlugLoadMBRTV,0.698630,0.704420,"{'199': [[['2014-01-16 05:59:19', '2014-01-16 06:29:19'], ['2014-01-16 06:14:19', '2014-01-16 06..."
44,Load_StatusSensHeatPrntBDOWN>Load_StatusBA2Lights,0.983562,0.991713,"{'199': [[['2014-01-16 17:59:45', '2014-01-16 21:59:44'], ['2014-01-16 20:14:45', '2014-01-16 20..."
51,Load_StatusSensHeatPrntBDOWN>Load_StatusBR2Lights,0.846575,0.853591,"{'199': [[['2014-01-16 17:59:45', '2014-01-16 21:59:44'], ['2014-01-16 18:29:45', '2014-01-16 21..."
54,Load_StatusSensHeatPrntBDOWN>Load_StatusBR3Lights,0.846575,0.853591,"{'199': [[['2014-01-16 17:59:45', '2014-01-16 21:59:44'], ['2014-01-16 18:29:45', '2014-01-16 20..."
69,Load_StatusSensHeatPrntBDOWN>Load_StatusPlugLoadBR2Laptop,0.986301,0.994475,"{'199': [[['2014-01-16 17:59:45', '2014-01-16 21:59:44'], ['2014-01-16 18:29:45', '2014-01-16 21..."
70,Load_StatusSensHeatPrntBDOWN>Load_StatusPlugLoadBR3Laptop,0.846575,0.853591,"{'199': [[['2014-01-16 17:59:45', '2014-01-16 21:59:44'], ['2014-01-16 18:29:45', '2014-01-16 20..."
274,Load_StatusSensHeatPrntADOWN>Load_StatusMBALights,0.849315,0.856354,"{'199': [[['2014-01-16 06:29:19', '2014-01-16 08:29:18'], ['2014-01-16 06:29:19', '2014-01-16 06..."
278,Load_StatusSensHeatPrntADOWN>Load_StatusBA2Lights,0.991781,1.000000,"{'199': [[['2014-01-16 06:29:19', '2014-01-16 08:29:18'], ['2014-01-16 06:29:19', '2014-01-16 06..."
282,Load_StatusSensHeatPrntADOWN>Load_StatusBR2Lights,0.849315,0.856354,"{'199': [[['2014-01-16 06:29:19', '2014-01-16 08:29:18'], ['2014-01-16 06:39:18', '2014-01-16 06..."


In [262]:
redundancy_filter_tool(dataframe=level2_df, regex_str='[\w]*\|[\w]*', select_index_dict='psn_app')

,pattern,supp,conf,time
156,Load_StatusPlugLoadMBRTV|Load_StatusSensHeatPrntADOWN,0.701370,0.707182,"{'199': [[['2014-01-16 06:14:19', '2014-01-16 06:59:18'], ['2014-01-16 06:29:19', '2014-01-16 08..."
264,Load_StatusPlugLoadLRTV|Load_StatusSensHeatChildAUP,0.701370,0.701370,"{'199': [[['2014-01-16 18:09:45', '2014-01-16 20:29:44'], ['2014-01-16 20:14:45', '2014-01-16 23..."
269,Load_StatusPlugLoadLRTV|Load_StatusSensHeatChildBUP,0.838356,0.838356,"{'199': [[['2014-01-16 18:09:45', '2014-01-16 20:29:44'], ['2014-01-16 20:14:45', '2014-01-16 23..."
301,Load_StatusSensHeatPrntADOWN|Load_StatusPlugLoadFan,0.698630,0.704420,"{'199': [[['2014-01-16 18:29:45', '2014-01-16 22:29:44'], ['2014-01-16 21:59:44', '2014-01-17 00..."
389,Load_StatusDRLights|Load_StatusSensHeatChildAUP,0.701370,0.701370,"{'199': [[['2014-01-16 19:14:45', '2014-01-16 20:29:44'], ['2014-01-16 20:14:45', '2014-01-16 23..."
390,Load_StatusDRLights|Load_StatusSensHeatChildBUP,0.701370,0.701370,"{'199': [[['2014-01-16 19:14:45', '2014-01-16 20:29:44'], ['2014-01-16 20:14:45', '2014-01-16 23..."
417,Load_StatusSensHeatChildBDOWN|Load_StatusBR2Lights,0.558904,0.563536,"{'258': [[['2014-03-16 12:58:31', '2014-03-16 18:28:31'], ['2014-03-16 18:18:31', '2014-03-16 23..."
489,Load_StatusSensHeatChildAUP|Load_StatusPlugLoadLRTV,0.975342,0.975342,"{'199': [[['2014-01-16 00:04:18', '2014-01-16 06:59:18'], ['2014-01-16 06:29:19', '2014-01-16 08..."
496,Load_StatusSensHeatChildAUP|Load_StatusDRLights,0.843836,0.843836,"{'199': [[['2014-01-16 18:29:45', '2014-01-16 19:29:45'], ['2014-01-16 19:14:45', '2014-01-16 20..."
508,Load_StatusSensHeatChildAUP|Load_StatusPlugLoadDesktopPCMonitor,0.698630,0.698630,"{'299': [[['2014-04-26 00:03:32', '2014-04-26 09:28:36'], ['2014-04-26 08:28:36', '2014-04-26 17..."
